In [1]:
# data to abstract as system arguments in script
interPickle = "data/pickled_targetedInteractions.pkl"
oligoPickle = "data/oligo_sequences.pkl"
path = "data/territory.bed"

In [2]:
import pandas as pd
# read pickles to save computation time
interactions = pd.read_pickle(interPickle)
bounds = pd.read_pickle(oligoPickle)

In [3]:
# # split data by territory covered, using the same criteria of chromosome + intersecting ranges
# # source: https://stackoverflow.com/questions/3269434/whats-the-most-efficient-way-to-test-two-integer-ranges-for-overlap
# i = 0
# total = 0
# territoryInteractions = {}
# for territory in bounds.itertuples():
#     t1 = territory[1]
#     t2 = territory[2]
#     territoryInteractions[i] = interactions.query("(X1 == 'chr8' & X2 <= @t2 & @t1 <= (X2 + 51)) | (X5 == 'chr8' & X6 <= @t2 &  @t1 <= (X6 + 51))", inplace=False)
#     total += territoryInteractions[i].shape[0]
#     i += 1
# print(total)

In [4]:
## separate interactor/interactee pairs and filter out territory regions. 
## subsequent concatenation and generation of ends from "start" of oligos yields easy conversion to .bed files
territoryInteractions = []
total = 0
for i in range(bounds.shape[0]):
    territory = bounds.iloc[i]
    t1 = territory[0]
    t2 = territory[1]
    i = territory.index
    df = interactions[((interactions["X1"] == 'chr8') & (interactions["X2"] <= t2) & (t1 <= (interactions["X2"] + 51))) | ((interactions["X5"] == 'chr8') & (interactions["X6"] <= t2) & (t1<=(interactions["X6"] + 51)))]
    #separation
    df1 = df.iloc[:,:2]
    df2 = df.iloc[:,2:]
    #filtering
    df2.columns = ['X1', 'X2']
    df1 = df1[(df1["X1"] != "chr8") | (df1["X2"] > t2) | (df1["X2"]+51 < t1)]
    df2 = df2[(df2["X1"] != "chr8") | (df2["X2"] > t2) | (df2["X2"]+51 < t1)]
    total += (df.shape[0] - df1.shape[0] - df2.shape[0])
    #concatenation
    df = pd.concat([df1, df2], sort=False)
    #end generation
    df["X2"] = (df["X2"]/5).round(decimals=-3)*5
    df["end"] = df["X2"] + 5000
    territoryInteractions.append(df)
print(total)

361


In [5]:
# generation of bed files
for i in range(len(territoryInteractions)):
    territoryInteractions[i].to_csv(path_or_buf=path[:-4] + "_" + str(i) + "_" + path[-4:], sep='\t', na_rep='', float_format="%.f", columns=None, header=False, index=False, index_label=None, mode='w', encoding=None, compression='infer', quoting=None, quotechar='"', line_terminator=None, chunksize=None, tupleize_cols=None, date_format=None, doublequote=True, escapechar=None, decimal='.')